In [76]:
import csv
import json
import mysql.connector

# please modify hw5_config.json to your own config file
with open('hw5_config.json') as f:
    config = json.load(f)

host = config['host']
user = config['user']
passwd = config['passwd']
dbms_student_path = config['dbms_student_path']
new_student_path = config['new_student_path']

# 連接到MySQL數據庫
conn = mysql.connector.connect(host=host, user=user, passwd=passwd)
cursor = conn.cursor(buffered=True)

In [77]:
cursor.execute("CREATE DATABASE DB_class")

cursor.execute("show databases")
print(cursor.fetchall())

cursor.execute("USE DB_class")

## 1-1: Input data into student table

In [78]:
# 創建學生表
cursor.execute('''
CREATE TABLE student (
    id INT AUTO_INCREMENT PRIMARY KEY,
    identity VARCHAR(50),
    department VARCHAR(100),
    degree INT,
    student_id VARCHAR(20),
    name VARCHAR(100),
    email VARCHAR(100),
    class VARCHAR(50)
    
);

''')

In [79]:
# 讀取CSV文件並插入數據
with open(dbms_student_path, 'r') as file:
    csv_reader = csv.reader(file)
    next(csv_reader)  # 跳過表頭
    for row in csv_reader:
        row[1] = row[1].strip() # 去除多餘空格
        cursor.execute('INSERT INTO student (identity, department, degree, student_id, name, email, class) VALUES (%s, %s, %s, %s, %s, %s, %s)',tuple(row))

conn.commit()

## 1-2: Read the information about yourself

In [80]:
# 查詢特定學生信息
cursor.execute('SELECT * FROM student WHERE student_id = %s', ('B09502132',))
result = cursor.fetchone()
print(result)

## 1-3: Update your own value of the "身份" column 

In [81]:
cursor.execute('UPDATE student SET identity = %s WHERE student_id = %s', ('特優生', 'B09502132'))
conn.commit()

In [82]:
# 查詢特定學生信息
cursor.execute('SELECT * FROM student WHERE student_id = %s', ('B09502132',))
result = cursor.fetchone()
print(result)

## 1-4: Insert three new students 

In [83]:
# 讀取CSV文件並插入數據
with open(new_student_path, 'r',encoding='utf-8') as file:
    csv_reader = csv.reader(file)
    next(csv_reader)  # 跳過表頭
    for row in csv_reader:
        cursor.execute('INSERT INTO student (identity, department, degree, student_id, name, email, class) VALUES (%s, %s, %s, %s, %s, %s, %s)',tuple(row))

conn.commit()

## 1-5: Prepared select statement 

In [84]:
ids =  ["R10123456", "B09987653", "R11123001"]  
query = "SELECT * FROM student WHERE student_id = %s"
for id in ids:
    cursor.execute(query, (id,))
    result = cursor.fetchone()
    print(result)

## 1-6: Group information

In [85]:
# 新增欄位
cursor.execute('ALTER TABLE student ADD COLUMN `group` INT DEFAULT 0, ADD COLUMN `captain` BOOLEAN DEFAULT 0')


group_ids = ['r12921059','r12922116','r12921093','b09502132']
for group_id in group_ids:
    cursor.execute('UPDATE student SET `group` = 7  WHERE student_id = %s', (group_id,))

# 更新組長
cursor.execute('UPDATE student SET `captain` = 1   WHERE student_id = %s',(group_ids[0],))  

conn.commit()

# 查詢自己組的成員信息
cursor.execute('SELECT * FROM student WHERE `group` = 7')
group_members = cursor.fetchall()
for member in group_members:
    print(member)


In [86]:
conn.close() # 關閉數據庫連接，不然執行 .sql 文件會卡住，無法執行